In [348]:
%config IPCompleter.greedy=True
import string
import re
import json
import csv
import os
from collections import defaultdict
import numpy as np
from pymystem3 import Mystem
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch

In [434]:
def json_read(filename):
    with open(filename, 'r') as inf:
        res = json.load(inf)
    return res

def json_dump(obj, filename, ea=False, indent=4):
    with open(filename, 'w') as ouf:
        json.dump(obj, ouf, ensure_ascii=ea, indent=indent)

In [413]:
class Span:
    def __init__(self, start, length, token, tag='NOTAG'):
        self.start = start
        self.end = start + length
        self.token = token
        self.tag = tag
        
    def set_tag(self, tag):
        self.tag = tag
        
    def __len__(self):
        return self.end - self.start
    
    def __repr__(self):
        return f'{self.token} {self.ltoken} {self.start}:{self.end} {self.tag}'

In [414]:
class Sentence:
    def __init__(self, sentence=None, spans=None):
        if sentence is not None:
            self.sentence = sentence
        if spans is not None:
            self.spans = spans
            
    def set_sentence(self, sentence):
        self.sentence = sentence
        
    def set_spans(self, spans):
        self.spans = spans
        
    def get_spans(self, tokenizer):
        self.spans = tokenizer(self.sentence)
        
    def lemmatize_spans(self, lemmatizer):
        for span in self.spans:
            span.ltoken = lemmatizer(span.token)
        
    def predict_tags(self, predictor):
        predictor(self.spans)
        
    def print_found_tags(self):
        for span in self.spans:
            if span.tag != 'NOTAG':
                print(span.token, span.tag)
            
    def get_found_tags(self):
        res = ''
        for span in self.spans:
            if span.tag != 'NOTAG':
                res += f'{span.start} {len(span)} {span.tag} '
        res += 'EOL'
        return res
        
    def __repr__(self):
        return self.sentence

In [415]:
train_sentences = []
with open('train_sentences.txt', 'r') as inf:
    for line in inf:
        train_sentences.append(Sentence(sentence=line.strip()))

In [416]:
with open('train_nes.txt') as inf:
    for i, line in enumerate(inf):
        spans_data = line.strip().split()
        spans = []
        for j in range(0, len(spans_data) - 1, 3):
            tag = spans_data[j + 2]
            start = int(spans_data[j])
            length = int(spans_data[j + 1])
            spans.append(Span(start, length, train_sentences[i].sentence[start:(start + length)], tag))
        train_sentences[i].set_spans(spans)
        train_sentences[i].lemmatize_spans(lemmatize)

### Data extraction

In [522]:
mystem = Mystem()
def lemmatize(token):
    ltoken = mystem.lemmatize(token)[0]
    if token[0].isupper():
        ltoken = ltoken.title()
    return ltoken

def lemmatize_text(text):
    tokens = text.split()
    ltokens = [lemmatize(token) for token in tokens]
    return ' '.join(ltokens)

In [374]:
persons_vocabulary = set()
for sent in train_sentences:
    for span in sent.spans:
        if span.tag == 'PERSON':
            if len(span.token) > 1:
                persons_vocabulary.add(lemmatize(span.token))

In [376]:
for file in os.listdir('Collection3'):
    if file.endswith('.ann'):
        with open(f'Collection3/{file}', 'r') as inf:
            for line in inf:
                _, span, text = line.strip().split('\t')
                tag = span.split()[0]
                tokens = re.findall(r'\w+', text)
                if tag == 'PER':
                    for token in tokens:
                        persons_vocabulary.add(lemmatize(token))

In [395]:
persons_vocabulary.remove('младший')
persons_vocabulary.remove('старший')
persons_vocabulary.remove('оборона')

#### Wikidata persons

In [441]:
with open('wikidata_persons.json', 'r') as inf:
    wikidata_persons = json.load(inf)

wikidata_persons_qids = []
for person in wikidata_persons:
    uri = person['subject']
    qid = uri.split('/')[-1]
    wikidata_persons_qids.append(qid)

wikidata_persons_qids_set = set(wikidata_persons_qids)

In [487]:
orgs_vocabulary = set()
for sent in train_sentences:
    for span in sent.spans:
        if span.tag == 'ORG':
            if len(span.token) > 2 and span.token[0].isupper():
                orgs_vocabulary.add(lemmatize(span.token))
# orgs_vocabulary.remove('0')
# orgs_vocabulary.remove('95')
orgs_vocabulary.remove('Россия')

In [488]:
orgs_vocabulary

{'Aar',
 'Abc',
 'Advisors',
 'Agence',
 'America',
 'Angeles',
 'Apple',
 'Auctions',
 'Bank',
 'Bbc',
 'Beatles',
 'Best',
 'Bfm',
 'Blekko',
 'Bloomberg',
 'Bonhams',
 'Border',
 'Business',
 'Börse',
 'Capital',
 'Caramba',
 'Cdma',
 'Channel',
 'Chappell',
 'Chatroulette',
 'Chrysler',
 'Citigroup',
 'Clearing',
 'Cleary',
 'Cme',
 'Cmea',
 'Companion',
 'Corp',
 'Corporation',
 'Corriere',
 'Cultural',
 'Customs',
 'Dell',
 'Depeche',
 'Deutsche',
 'Digital',
 'Disney',
 'Dst',
 'Economist',
 'Elevation',
 'Emc',
 'Euronext',
 'Exchanges',
 'Exor',
 'Facebook',
 'Fashion',
 'Fia',
 'Fiat',
 'Fide',
 'Five',
 'Flashpoint',
 'Forbes',
 'Fotoloft',
 'France',
 'Fremantle',
 'G',
 'G20',
 'Gazzetta',
 'Global',
 'Google',
 'Gottlieb',
 'Greenfieldproject',
 'Group',
 'Guardian',
 'Gzt',
 'Hamilton',
 'Harles',
 'Heritage',
 'Hermitage',
 'Hewlett',
 'Hong',
 'Ice',
 'Idc',
 'Informatics',
 'Inteligencji',
 'Intelligence',
 'Ivi',
 'Jentzsch',
 'Katolickiej',
 'Kinoklub',
 'Klub',
 'K

In [510]:
orgs_vocabulary_tuned = set()
for sent in train_sentences:
    i = 0
    while i < len(sent.spans):
        if sent.spans[i].tag == 'ORG':
            j = i + 1
            while j < len(sent.spans) and sent.spans[j].tag == 'ORG':
                j += 1
            org_tokens = [span.ltoken for span in sent.spans[i:j]]
            orgs_vocabulary_tuned.add(' '.join(org_tokens))
            i = j
        i += 1

In [524]:
for file in os.listdir('Collection3'):
    if file.endswith('.ann'):
        with open(f'Collection3/{file}', 'r') as inf:
            for line in inf:
                _, span, text = line.strip().split('\t')
                tag = span.split()[0]
                tokens = re.findall(r'\w+', text)
                if tag == 'ORG':
                    orgs_vocabulary_tuned.add(' '.join(lemmatize(t) for t in tokens))

In [525]:
orgs_vocabulary_tuned

{'Газпром экспорт',
 'Промсвязьбанк',
 'Национальный антикоррупционный комитет',
 'красный',
 'Государственный новостной агентство Северный Корея Kcna',
 'российский телекоммуникационный оператор компания с государственный капитал Ростелеком Cdma Скай Линк Скай Линка',
 'Московский государственный педагогический университет',
 'Рбк Информационный система',
 'En Management Llc',
 'Федеральный налоговый служба Фнс',
 'Роснефть Bank of America Merrill Lynch Bank of America Citigroup Global Markets Ltd',
 'Тамбовский областной онкологический диспансер',
 'советский консульство',
 'Военторг',
 'Фатх',
 'Эхо Москва радиостанция',
 'Российский экономический академия они Г В Плеханова',
 'оргкомитет Олимпийский игра в Сочи',
 'Секретариат Кабинет Министр',
 'Ооо Мечесть Энерго',
 'Blitz Quotidiano',
 'Hr департамент',
 'Ассоциация пенсионный фонд',
 'Американский космический агентство',
 'Войско Польский',
 'британский газета Mail on Sunday московский антидопинговый лаборатория',
 'Центр общес

In [293]:
with open('orgs_vocabulary.json', 'w') as ouf:
    json.dump(list(orgs_vocabulary), ouf, ensure_ascii=False, indent=4)

### Prediction

In [379]:
def tokenizer(sentence):
    matches = list(re.finditer(r'\w+', sentence))
    spans = []
    for match in matches:
        start = match.start()
        length = match.end() - start
        token = match[0]
        spans.append(Span(start, length, token))
    return spans

In [380]:
def is_latin_word(token):
    return token[0] in string.ascii_letters

In [381]:
def is_vocab_person(ltoken):
    return ltoken in persons_vocabulary

def is_person(ltoken):
    return is_vocab_person(ltoken)

def is_wikidata_person(text):
    query = {
        'query': {
            'bool': {
                'should': [
                    {
#                         'match': {
#                             'label': {
#                                 'query': text,
#                                 "fuzziness": 'AUTO',
#                                 "prefix_length": 1,
#                                 'fuzzy_transpositions': False
#                             }
#                         }
                        'match_phrase': {
                            'label': text
                        }
                    }
                ]
            }
        }
    }
    res = es.search(index='label_single', body=query, size=10)
    hits = res['hits']['hits']
    qids = sorted([hit['_source']['qid'] for hit in hits], key=lambda s: int(s[1:]))
    for qid in qids[:1]:
        if qid in wikidata_persons_qids_set:
            return True
    return False

In [561]:
def is_latin_org(tokens):
    if len(tokens) < 3:
        return False
    if tokens[0][0].islower():
        return False
    for token in tokens:
        if not is_latin_word(token):
            return False
    return True

def is_wikidata_org_type(ltokens):
    text = ' '.join(ltokens)
    return text in wikidata_org_types_vocabulary

def is_wikidata_org(ltokens):
    text = ' '.join(ltokens)
    return (text in wikidata_orgs_vocabulary) or (text in wikidata_all_orgs_vocabulary)

def is_vocabulary_org(ltokens):
    text = ' '.join(ltokens)
    return text in orgs_vocabulary_tuned

def is_in_org_vocabulary(ltokens):
    text = ' '.join(ltokens)
    return text in orgs_vocabulary

def is_org(tokens, ltokens):
    if len(tokens) > 10:
        return False
    return is_latin_org(tokens) or is_wikidata_org_type(ltokens) or is_wikidata_org(ltokens) or is_vocabulary_org(ltokens)

In [562]:
def tag_predictor(spans):
    # orgs
    for i in range(len(spans)):
        for j in range(i, len(spans)):
            tokens = [span.token for span in spans[i:(j + 1)]]
            ltokens = [span.ltoken for span in spans[i:(j + 1)]]
            if is_org(tokens, ltokens):
                for k in range(i, j + 1):
                    spans[k].tag = 'ORG'
                for k in range(j + 1, len(spans)):
                    if spans[k].token[0].isupper():
                        spans[k].tag = 'ORG'
                    else:
                        break
                        
    
    
    
    # wikidata persons
    for i in range(len(spans)):
        cur_token = spans[i].token
        if i != len(spans) - 1:
            next_token = spans[i + 1].token
        else:
            next_token = 'lol'
        if cur_token[0].isupper() and next_token[0].isupper():
            text = cur_token + ' ' + next_token
            if is_wikidata_person(text):
                spans[i].tag = 'PERSON'
                spans[i + 1].tag = 'PERSON'
        elif cur_token[0].isupper() and i > 0 and len(cur_token) > 1 and is_wikidata_person(cur_token):
            spans[i].tag = 'PERSON'
            
                
    # known persons
    for i, span in enumerate(spans[1:], start=1):
        token = span.token
        ltoken = span.ltoken
        if is_person(ltoken):
            span.tag = 'PERSON'
            if i < len(spans) - 1:
                next_token = spans[i + 1].token
                if next_token[0].isupper() and spans[i + 1].tag == 'NOTAG':
                    spans[i + 1].tag = 'PERSON'

In [486]:
w = 'искусства'
print(lemmatize(w))
lemmatize(w) in orgs_vocabulary

искусство


True

In [565]:
with open('dataset_40163_1.txt', 'r') as inf, open('output.txt', 'w') as ouf, open('watch.txt', 'w') as wouf:
    for line in tqdm(inf):
        sentence = Sentence(line.strip())
        sentence.get_spans(tokenizer)
        sentence.lemmatize_spans(lemmatize)
        sentence.predict_tags(tag_predictor)
        sentence.print_found_tags()
        for span in sentence.spans:
            print(span, file=wouf)
        print(sentence.get_found_tags(), file=ouf)

США ORG
группы ORG
Минске ORG
ДНР ORG
ЛНР ORG
Мердока PERSON
издательства ORG
News ORG
International ORG
Об ORG
ГАБТа ORG
Владимир PERSON
Урин PERSON
театров ORG
Комиссия ORG
3 ORG
Гребенников PERSON
ведомства ORG
ЗАЗ ORG
Таврия ORG
Украине ORG
Бориса PERSON
Гребенщикова PERSON
Японию ORG
Харуки PERSON
Мураками PERSON
Россию ORG
войска ORG
войска ORG
Донецке ORG
Венедиктов PERSON
Владимиру PERSON
Путину PERSON
СМИ ORG
совет ORG
директоров ORG
Эха ORG
Москвы ORG
бар ORG
Хайнекен PERSON
Телониуса PERSON
Монка PERSON
Спартаком ORG
Локомотив ORG
интернет ORG
НАСА ORG
Роскосмоса ORG
Союз ORG
ТМ ORG
Союз ORG
ТМ ORG
15 ORG
Они ORG
Ботсваны ORG
страной ORG
The ORG
Beatles ORG
Великобританию ORG
Вместе ORG
МИД ORG
РФ ORG
Москвы ORG
Токио ORG
Сергей PERSON
Тивяков PERSON
Нидерланды ORG
3 ORG
Бернс ORG
Сен PERSON
Луи ORG
Хитли PERSON
ГБ ORG
Regnum ORG
агентства ORG
холдингу ORG
Газпром ORG
МИД ORG
Литвы ORG
Линас ORG
Линкявичус ORG
Москву ORG
независимых ORG
государств ORG
Если ORG
Киев ORG
Газпр

Соединенные ORG
Штаты ORG
Андрею PERSON
Санникову PERSON
Беларуси ORG
Зрители ORG
Иисуса PERSON
Христа PERSON
Венеры PERSON
Без ORG
Крыма ORG
Наш ORG
Если ORG
город ORG
город ORG
город ORG
издания ORG
Fierce ORG
Biotech ORG
Джон PERSON
Кэррол PERSON
Шкрели PERSON
России ORG
Харуки PERSON
Мураками PERSON
сотрудничества ORG
странами ORG
Реджеп PERSON
Эрдоган PERSON
стране ORG
Азии ORG
Дальнего ORG
Востока ORG
Кремль ORG
Министерства ORG
обороны ORG
Украины ORG
Виктора PERSON
Януковича PERSON
Он ORG
интернет ORG
сборной ORG
команды ORG
агентству ORG
News ORG
of ORG
the ORG
World ORG
Невилл PERSON
Терлбек ORG
Джеймс PERSON
Уитерап PERSON
Грег PERSON
Мискив PERSON
Сегодня ORG
Телевизор ORG
интернет ORG
Tumblr ORG
Топалов PERSON
Он ORG
Украины ORG
красной ORG
ЗАО ORG
СК ORG
Ритейл ORG
О PERSON
ООО ORG
Фрут ORG
Сервис ORG
Чикаго ORG
КНР ORG
Ху PERSON
Цзиньтао PERSON
США ORG
По ORG
магазинов ORG
Савеловский ORG
суд ORG
Москвы ORG
телеканала ORG
Дождь ORG
Гагаузии ORG
Мормоттана PERSON
Толстого

In [106]:
sent = Sentence(sentence='В общей сложности на аукцион выставляют 300 лотов, имеющих отношение к The Beatles и входящих в коллекцию, собранную германским продюсером Уве Блашке.')
sent.get_spans(tokenizer)
sent.predict_tags(tag_predictor)
print(sent.spans)

[В 0:1 NOTAG, общей 2:7 NOTAG, сложности 8:17 NOTAG, на 18:20 NOTAG, аукцион 21:28 NOTAG, выставляют 29:39 NOTAG, 300 40:43 NOTAG, лотов 44:49 NOTAG, имеющих 51:58 NOTAG, отношение 59:68 NOTAG, к 69:70 NOTAG, The 71:74 NOTAG, Beatles 75:82 NOTAG, и 83:84 NOTAG, входящих 85:93 NOTAG, в 94:95 NOTAG, коллекцию 96:105 NOTAG, собранную 107:116 NOTAG, германским 117:127 NOTAG, продюсером 128:138 NOTAG, Уве 139:142 PESRON, Блашке 143:149 PESRON]


### Exploration

In [505]:
mystem = Mystem()

In [54]:
for sent in train_sentences:
    for span in sent.spans:
        if span.tag == 'PERSON':
            first_letter = span.token[0]
            if first_letter.islower():
                print(span.token)

аль-Хадж
al-Haj
аль-Барадеи
tiphareth
фон
эль-Масри
el-Masri
де
де
де


In [69]:
persons_vocabulary = set()
for sent in train_sentences:
    for span in sent.spans:
        if span.tag == 'PERSON':
            if len(span.token) > 1:
                persons_vocabulary.add(mystem.lemmatize(span.token)[0].lower())
persons_vocabulary.remove('ли')
persons_vocabulary.remove('о')

In [70]:
list(filter(lambda s: len(s) < 3, persons_vocabulary))

['си',
 'де',
 'ле',
 'ир',
 'ги',
 'ху',
 'el',
 'ши',
 'бг',
 'су',
 'еп',
 'со',
 'ус',
 'ён',
 'al']

In [142]:
orgs_vocabulary = set()
for sent in train_sentences:
    for span in sent.spans:
        if span.tag == 'ORG':
            if len(span.token) > 2 and span.token[0].isupper():
                orgs_vocabulary.add(mystem.lemmatize(span.token)[0].lower())
# orgs_vocabulary.remove('0')
# orgs_vocabulary.remove('95')
# orgs_vocabulary.remove('ю')
# orgs_vocabulary.remove('s')
# orgs_vocabulary.remove('u')

In [141]:
orgs_vocabulary

{'aar',
 'abc',
 'advisors',
 'agence',
 'america',
 'angeles',
 'apple',
 'auctions',
 'bank',
 'bbc',
 'beatles',
 'best',
 'bfm',
 'blekko',
 'bloomberg',
 'bonhams',
 'border',
 'business',
 'börse',
 'capital',
 'caramba',
 'cdma',
 'channel',
 'chappell',
 'chatroulette',
 'chrysler',
 'citigroup',
 'clearing',
 'cleary',
 'cme',
 'cmea',
 'companion',
 'corp',
 'corporation',
 'corriere',
 'cultural',
 'customs',
 'dell',
 'depeche',
 'deutsche',
 'digital',
 'disney',
 'dst',
 'economist',
 'elevation',
 'emc',
 'euronext',
 'exchanges',
 'exor',
 'facebook',
 'fashion',
 'fia',
 'fiat',
 'fide',
 'five',
 'flashpoint',
 'forbes',
 'fotoloft',
 'france',
 'fremantle',
 'g',
 'g20',
 'gazzetta',
 'global',
 'google',
 'gottlieb',
 'greenfieldproject',
 'group',
 'guardian',
 'gzt',
 'hamilton',
 'harles',
 'heritage',
 'hermitage',
 'hewlett',
 'hong',
 'ice',
 'idc',
 'informatics',
 'inteligencji',
 'intelligence',
 'ivi',
 'jentzsch',
 'katolickiej',
 'kinoklub',
 'klub',
 'k

In [198]:
cnt = 0
with open('names/russian_names.json', 'r') as inf:
    cur_names = json.load(inf)
    for name in cur_names:
        if name['PeoplesCount'] > 200000:
            print(name['Name'])
            cnt += 1

Азия
Александра
Александрия
Алена
Алёна
Алина
Алиса
Алла
Алма
Альберта
Альбина
Анастасия
Ангелина
Анжелика
Анна
Анюта
Аня
Аргентина
Армения
Атланта
белая
Валенсия
Валентина
Валерия
Вера
Вероника
Виктория
Вирджиния
Галина
Дания
Дарья
Даша
Диана
Евгения
Екатерина
Елена
Елизавета
Жанна
Женя
Ивано
Индиана
Инна
Ира
Ирина
Карелия
Карина
Каролина
Катарина
Катерина
Кристина
Ксения
Ланка
Лариса
Лена
Ленина
Лера
Лидия
Лиза
Лилия
Любовь
Людмила
Маргарита
Марина
Мария
Маша
Надежда
Настя
Натали
Наталия
Наталья
Наташа
Нина
Оксана
Олеся
Ольга
Оля
Паша
Петра
Полина
Рика
Россия
Санта
Саша
Света
Светлана
Севилья
Сербия
Сицилия
София
Стамбул
Тамара
Таня
Татьяна
слава
Флорида
Химико
Эльвира
Эмилия
Юлия
Юля
Юта
Ява
Яна
Алания
Аризона
Бали
Барселона
Бостон
Бразилия
Вика
Гана
Голландия
Дакота
Дели
Дима
Европа
Иль
Индия
Ирландия
Италия
Кали
Калифорния
Канада
Канзас
Катя
Кения
Кентукки
Колумбия
Кострома
Ксюша
Лима
Лондон
Мир
Миша
Невада
Никита
Ниу
Нью
Одесса
Рейн
Рига
Рико
Рио
Рома
Самара
Сан
Саха
Сидней
Униве

In [199]:
cnt

292

In [188]:
mystem.lemmatize('Мирас')

['мирас', '\n']

In [189]:
'мир' in names_vocabulary

True

In [457]:
wikidata_businesses = json_read('wikidata_businesses.json')
wikidata_companies = json_read('wikidata_companies.json')

In [458]:
wikidata_org_types = [t['item'].split('/')[-1] for t in wikidata_businesses] + [t['item'].split('/')[-1] for t in wikidata_companies]

In [459]:
wikidata_org_types[0]

'Q4830453'

In [460]:
wikidata_org_types_vocabulary = set()
for qid in tqdm(wikidata_org_types):
    doc = {
        'query': {
            'bool': {
                'must': [
                    {
                        'match_phrase': {
                            'qid': qid
                        }
                    }
                ]
            }
        }
    }
    res = es.search(index='label_single', body=doc, size=1000)
    for hit in res['hits']['hits']:
        label = hit['_source']['label']
        label = ' '.join(lemmatize(t) for t in re.findall(r'\w+', label))
        wikidata_org_types_vocabulary.add(label)

In [461]:
wikidata_org_types_vocabulary

{'',
 'Aok',
 'Bed and breakfast',
 'Cactus',
 'Carsharing',
 'Dotcom',
 'Farmers market',
 'Gmbh',
 'Honky tonk',
 'Mavag',
 'Mbh',
 'Mávag',
 'Pub',
 'Public house',
 'Rss агрегатор',
 'Safeway',
 'Samizdat',
 'Spa',
 'Spa курорт',
 'Spa салон',
 'Starbucks',
 'Steakhouse',
 'Tesco',
 'Trader Joe s',
 'Veb',
 'besloten vennootschap met beperkte aansprakelijkheid',
 'diner',
 'private banking',
 'public limited company',
 'tea house',
 'Автодилер',
 'Автомобильный дилер',
 'Автосвалка',
 'Бесплатный столовая',
 'Библейский общество',
 'Биргартен',
 'Блошиный рынок',
 'Больший тройка лейбл звукозапись',
 'Больший четверка лейбл звукозапись',
 'Брассери',
 'Брокеридж',
 'Бульварный газета',
 'Бункер подводный лодка',
 'Бюджетный авиакомпания',
 'Бюро путешествие',
 'Вагон ресторан',
 'Венский кофейня',
 'Венчура',
 'Венчурный проект',
 'Виноградник',
 'Гипермаркет',
 'Госпредприятие',
 'Гостиный двор',
 'Гость ный дво р',
 'Государственный компания',
 'Деликатес',
 'Дзайбацу',
 'Дискоте

In [496]:
wikidata_businesses = json_read('wikidata_businesses.json')
wikidata_companies = json_read('wikidata_companies.json')

In [507]:
len(wikidata_companies)

91145

In [500]:
wikidata_orgs = [t['item'].split('/')[-1] for t in wikidata_businesses] + [t['item'].split('/')[-1] for t in wikidata_companies]

In [508]:
wikidata_orgs_set = set(wikidata_orgs)

In [509]:
wikidata_orgs_vocabulary = set()
for qid in tqdm(wikidata_orgs_set):
    doc = {
        'query': {
            'bool': {
                'must': [
                    {
                        'match_phrase': {
                            'qid': qid
                        }
                    }
                ]
            }
        }
    }
    res = es.search(index='label_single', body=doc, size=1000)
    for hit in res['hits']['hits']:
        label = hit['_source']['label']
        label = ' '.join(lemmatize(t) for t in re.findall(r'\w+', label))
        wikidata_orgs_vocabulary.add(label)

In [539]:
wikidata_orgs_vocabulary

{'',
 'Калининградский железный дорога',
 'Клайдсдейл банк',
 'Научно исследовательский институт имя В В Тихомирова',
 'Air astana',
 'Сибмост',
 'Graham Paige',
 'Персидский музыкальный метка',
 'Монтон спортс',
 'Filmlari',
 'South African Breweries',
 'Gakken',
 'Xom',
 'Университет Ланкастер',
 'Cned Национальный центр дистанционный образование',
 'Юрайт',
 'Capitol Records',
 'Bismillah Airlines',
 'Мусороперерабатывающий завод',
 'Multinex',
 'Next Model Management',
 'Баварский государственный опера',
 'Stada Cis',
 'Клозери де Лить',
 'Kunming Airlines',
 'Институт Земля Колумбийский университет',
 'Three Rivers Computer Corporation',
 'American Institute of Physics',
 'Почта Мавритания',
 'Кристмас Филмза',
 'Университет Белград',
 'Регнум ру',
 'Вибер',
 'Ярославский государственный автомобильный завод',
 'Viessmann',
 'Excalibur Almaz',
 'Пензенский завод вычислительный техника',
 'Сибирский научно исследовательский институт авиация имя С А Чаплыгин',
 'Atlantic Petroleum',


In [553]:
wikidata_all_orgs = json_read('wikidata_orgs.json')

In [556]:
wikidata_all_orgs = [t['item'].split('/')[-1] for t in wikidata_all_orgs]

In [557]:
wikidata_all_orgs_set = set(wikidata_all_orgs)

In [559]:
wikidata_all_orgs_vocabulary = set()
for qid in tqdm(wikidata_all_orgs_set):
    doc = {
        'query': {
            'bool': {
                'must': [
                    {
                        'match_phrase': {
                            'qid': qid
                        }
                    }
                ]
            }
        }
    }
    res = es.search(index='label_single', body=doc, size=100)
    for hit in res['hits']['hits']:
        label = hit['_source']['label']
        label = ' '.join(lemmatize(t) for t in re.findall(r'\w+', label))
        wikidata_all_orgs_vocabulary.add(label)

In [560]:
wikidata_all_orgs_vocabulary

{'',
 'Грос Энгерсдорф',
 'Алматинский технологический институт',
 'Руск Поток',
 'Another Messiah',
 'Сено Фортюн сюр Эйрье',
 'Катингейр',
 'Сарсуэла муниципалитет',
 'Обнинский школа шесть',
 'Вехнянский волость',
 'Earth Fire',
 'Кастельгульельмо',
 'Absurd',
 'Группа Орли',
 'Святой Никольский женский монастырь г Кирово Чепецк',
 'Синсиро',
 'Кальтенкирхен',
 'Filmlari',
 'Бернулли семья',
 'Сальдивиа',
 'Ассоциация шведский публицист',
 'Synapscape',
 'Румелэнг',
 'Молотковичский сельсовет',
 'Форня Авольтри',
 'Skambankt',
 'Уижа',
 'Сан Бенедетто дей Марси',
 'Агентство по охрана окружающий среда Сша',
 'Курскгту',
 'Бгай',
 'Next Model Management',
 'Take That',
 'Michigan State University',
 'Усть Катавский городской округ',
 'Сборная Тунис по регби',
 'Жальгирис',
 'Рвваиокзу',
 'Архентинос Хуниорс',
 'Шолс',
 'Silly Wizard',
 'Салпетриер',
 'Лиммен',
 'Национальный радио Чувашия',
 'Wsa Pushbikers',
 'Британский Содружество',
 'Городской округ Калуга',
 'Сено Бонн ан Шансор

In [203]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360, 'maxsize': 25}])

In [337]:
text = 'ICE'
doc = {
    'query': {
        'bool': {
            'should': [
                {
#                     'match': {
#                         'label': {
#                             'query': text,
#                             "fuzziness": 'AUTO',
#                             "prefix_length": 1,
#                             'fuzzy_transpositions': False
#                         }
#                     }
                    'match_phrase': {
                        'label': text
                    }
                }
            ]
        }
    }
}
res = es.search(index='label_single', body=doc, size=10)
res['hits']

{'total': {'value': 68, 'relation': 'eq'},
 'max_score': 14.384574,
 'hits': [{'_index': 'label_single',
   '_type': '_doc',
   '_id': 'DSfypmwBg_1FN09Ht6DV',
   '_score': 14.384574,
   '_source': {'qid': 'Q37549494', 'label': 'Ice'}},
  {'_index': 'label_single',
   '_type': '_doc',
   '_id': 'Cu7vpmwBg_1FN09H_5xi',
   '_score': 13.880581,
   '_source': {'qid': 'Q2389029', 'label': 'Ice Ice Baby'}},
  {'_index': 'label_single',
   '_type': '_doc',
   '_id': 'RdzvpmwBg_1FN09HE-7O',
   '_score': 11.5399475,
   '_source': {'qid': 'Q157534', 'label': 'Black Ice'}},
  {'_index': 'label_single',
   '_type': '_doc',
   '_id': 'YN3vpmwBg_1FN09HGEx0',
   '_score': 11.5399475,
   '_source': {'qid': 'Q173637', 'label': 'Ice Cube'}},
  {'_index': 'label_single',
   '_type': '_doc',
   '_id': 'EN_vpmwBg_1FN09HMlSY',
   '_score': 11.5399475,
   '_source': {'qid': 'Q309941', 'label': 'Ice T'}},
  {'_index': 'label_single',
   '_type': '_doc',
   '_id': 'BN_vpmwBg_1FN09HNHYK',
   '_score': 11.5399475

In [440]:
qid = wikidata_org_types[0]
doc = {
    'query': {
        'bool': {
            'must': [
                {
                    'match_phrase': {
                        'qid': qid
                    }
                }
            ]
        }
    }
}
res = es.search(index='label_single', body=doc, size=1000)
res['hits']

{'total': {'value': 7, 'relation': 'eq'},
 'max_score': 13.492668,
 'hits': [{'_index': 'label_single',
   '_type': '_doc',
   '_id': 'PNrupmwBg_1FN09H9Y35',
   '_score': 13.492668,
   '_source': {'qid': 'Q43229', 'label': 'корпорация'}},
  {'_index': 'label_single',
   '_type': '_doc',
   '_id': 'P9rupmwBg_1FN09H9Y35',
   '_score': 13.492668,
   '_source': {'qid': 'Q43229', 'label': 'предприятие'}},
  {'_index': 'label_single',
   '_type': '_doc',
   '_id': 'OtrupmwBg_1FN09H9Y35',
   '_score': 13.492668,
   '_source': {'qid': 'Q43229', 'label': 'объединение'}},
  {'_index': 'label_single',
   '_type': '_doc',
   '_id': 'O9rupmwBg_1FN09H9Y35',
   '_score': 13.492668,
   '_source': {'qid': 'Q43229', 'label': 'компания'}},
  {'_index': 'label_single',
   '_type': '_doc',
   '_id': 'PdrupmwBg_1FN09H9Y35',
   '_score': 13.492668,
   '_source': {'qid': 'Q43229', 'label': 'учреждение'}},
  {'_index': 'label_single',
   '_type': '_doc',
   '_id': 'PtrupmwBg_1FN09H9Y35',
   '_score': 13.492668

In [336]:
is_wikidata_person('ICE')

True

In [265]:
string.

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [351]:
with open('Collection3/1040.ann', 'r') as inf:
    for line in inf:
        print(line.strip().split('\t'))

['T1', 'PER 0 8', 'Игнатьев']
['T2', 'ORG 25 33', 'Минфином']
['T3', 'ORG 83 91', 'Минфином']
['T4', 'LOC 92 94', 'РФ']
['T5', 'ORG 183 185', 'ЦБ']
['T6', 'PER 186 201', 'Сергей Игнатьев']
['T7', 'ORG 279 287', 'Минфином']
['T8', 'LOC 288 290', 'РФ']
['T9', 'ORG 379 381', 'ЦБ']
['T10', 'PER 382 397', 'Сергей Игнатьев']
['T11', 'ORG 410 417', 'Госдуме']
['T12', 'ORG 432 439', 'Минфина']
['T13', 'PER 440 454', 'Антон Силуанов']
['T14', 'ORG 805 811', 'Минфин']
['T15', 'ORG 857 859', 'ЦБ']
['T16', 'PER 1125 1133', 'Игнатьев']
['T17', 'ORG 1157 1169', 'Банка России']
['T18', 'LOC 1233 1235', 'РФ']
['T19', 'PER 1396 1404', 'Игнатьев']
['T20', 'ORG 1695 1702', 'Минфина']
['T21', 'PER 1750 1758', 'Игнатьев']
['T22', 'PER 1806 1826', 'Антоном Германовичем']
['T23', 'LOC 1845 1847', 'РФ']
['T24', 'PER 1848 1856', 'Силуанов']
['T25', 'ORG 1948 1959', 'Центробанка']
['T26', 'ORG 2002 2009', 'Минфина']
['T27', 'ORG 2038 2045', 'Минфина']
['T28', 'LOC 2046 2048', 'РФ']
['T29', 'ORG 2310 2318', 'Мин

In [364]:
collection3_names = set()
collection3_orgs = set()
for file in os.listdir('Collection3'):
    if file.endswith('.ann'):
        with open(f'Collection3/{file}', 'r') as inf:
            for line in inf:
                _, span, text = line.strip().split('\t')
                tag = span.split()[0]
                tokens = re.findall(r'\w+', text)
                if tag == 'PER':
                    for token in tokens:
                        collection3_names.add(token)
                if tag == 'ORG':
                    for token in tokens:
                        collection3_orgs.add(token)

In [365]:
collection3_names

{'Бизоньеро',
 'Чистяков',
 'Джулиане',
 'Велихова',
 'Хугаева',
 'Адриано',
 'Ван',
 'Wayne',
 'Козлова',
 'Тямушкину',
 'Меликьян',
 'Ильницкий',
 'Рашкину',
 'Годвин',
 'Хаффингтон',
 'Бреннана',
 'Морозова',
 'Маккристала',
 'Голиковой',
 'Васильевич',
 'Бородастова',
 'Набоков',
 'Владимирович',
 'Буксмана',
 'Фрейберга',
 'Жигунов',
 'Числов',
 'Аллиот',
 'Кристофер',
 'Грибоедова',
 'Горкин',
 'Малкер',
 'Айссата',
 'Назарбаева',
 'Гаева',
 'Пола',
 'Мозы',
 'Караваев',
 'Ingvar',
 'Крина',
 'Кромвелл',
 'Ромпей',
 'Юдина',
 'Юньшань',
 'Рамазан',
 'Гюль',
 'МакКиннан',
 'Боташев',
 'Сиваков',
 'Гасанов',
 'Абдельазиз',
 'Рошалю',
 'Вадиму',
 'Джебран',
 'Мен',
 'Шерин',
 'Владимировича',
 'Гудкова',
 'Бокерия',
 'Мажарова',
 'Вивиан',
 'Сидоренко',
 'Боженов',
 'Смиту',
 'Эхуда',
 'Karzai',
 'Карпель',
 'Theresa',
 'Нам',
 'Меркушкин',
 'Сункара',
 'Мизулина',
 'Чернакова',
 'Козлову',
 'Мердок',
 'Рольдос',
 'Аничин',
 'Дулькин',
 'Кэтлин',
 'Мифтахов',
 'Белова',
 'Булушева',

In [ ]:
enumerate()